# io power - Samna only - v0.39.12

- Use PDM_CLK and PDM_DATA driven by FPGA to XA3:

1. Keep both PDM_DATA and PDM_CLK low：(0x2b,1),(0x2d,0),(0x2c,0)
2. Keep PDM_DATA high and PDM_CLK low: (0x2b,1),(0x2d,0),(0x2c,1)
3. Keep PDM_DATA low and PDM_CLK high: (0x2b,1),(0x2d,0),(0x2c,2)
4. Keep PDM_DATA high and PDM_CLK high: (0x2b,1),(0x2d,0),(0x2c,3)
5. Toggle PDM_DATA at some frequency while keeping PDM_CLK low:  (0x2b,1),(0x2d,1),(0x2c,0)
6. Toggle PDM_DATA at some frequency while keeping PDM_CLK high: (0x2b,1),(0x2d,1),(0x2c,1)
7. Keep PDM_DATA low while toggling PDM_CLK at some frequency: (0x2b,1),(0x2d,2),(0x2c,0)
8. Keep PDM_DATA high while toggling PDM_CLK at some frequency: (0x2b,1),(0x2d,2),(0x2c,2)
9. Toggle both PDM_CLK and PDM_DATA at some frequency: (0x2b,1),(0x2d,3)

### Initialization and connection to the board

In [1]:
import samna
print(f'Samna version {samna.__version__}')
import numpy as np

# Open the device and connect it to the power monitor.
xylo_node = samna.device.open_device("XyloAudio3TestBoard")

power_monitor = xylo_node.get_power_monitor()
sink_pm = samna.graph.sink_from(power_monitor.get_source_node())
stopwatch = xylo_node.get_stop_watch()
io = xylo_node.get_io_module()

xylo_model = xylo_node.get_model()
source = samna.graph.source_to(xylo_model.get_sink_node())

# We are only interested in Readout events, so we make a filter graph to filter only these events.
# Important note: `graph` needs to be kept alive for the filter to work.
graph = samna.graph.EventFilterGraph()
_, etf, readout_sink = graph.sequential([xylo_model.get_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
etf.set_desired_type('xyloAudio3::event::Readout')

# XyloConfiguration
xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()

## With the network, clock frequency can not go low as 6.25
clock_frequencies = [12.5, 25, 50]
ts = 1e-3

power_measurements_625 = []
power_measurements_125 = []
power_measurements_25 = []
power_measurements_50 = []

Samna version 0.39.12.1+g3f321cfa


### Configure PDM_CLK in 1 MHz and MAIN_CLK

In [2]:
def configure_clocks():

    if (enable_main_clock):
        # Defines PDM clock rate
        config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
        config.main_clock_frequency = 6250000
        config.pdm_clock_frequency = 1000000
        xylo_node.reset_board_soft(config)
    
    else:
        # Defines PDM clock rate
        config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
        config.pdm_clock_frequency = 1000000
        xylo_node.reset_board_soft(config)

        # Disable main clock
        io.write_config(0x0008, 0)


def configure_clock_direction():
    if pdm_clock == "fpga":
        print("fpga -> xylo")
        xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0
    else:
        print("xylo -> fpga")
        xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1


### Use PDM_CLK and PDM_DATA driven by FPGA:
- No network mapped, and main clock is disabled
  


In [3]:
enable_main_clock = True
pdm_clock = "fpga"

### Read register

In [4]:
def read_register(addr):
    source.write([samna.xyloAudio3.event.ReadRegisterValue(address = addr)])
    events = sink.get_n_events(1, 3000)
    assert(len(events) == 1)
    return events[0].data

ctrl1 = 0x0001
ctrl2 = 0x0002
ctrl3 = 0x0003
tr_wrap = 0x0004
hm_tr_wrap = 0x0005
clk_ctrl = 0x0006
clk_div = 0x0007
pwr_ctrl1 = 0x0008
pwr_ctrl2 = 0x0009
pwr_ctrl3 = 0x000A
pwr_ctrl4 = 0x000B
pad_ctrl = 0x000C
ie1 = 0x000E
ie2 = 0x000F
out_ctrl = 0x0011
monsel = 0x0166
mon_grp_sel = 0x0167
dbg_ctrl1 = 0x0168
dbg_stat1 = 0x0171
dfe_ctrl = 0x001B
ivgen = 0x0015
ivgen2 = 0x0016
ivgen3 = 0x0017
ivgen4 = 0x0018
ivgen5 = 0x0019
ivgen6 = 0x001A
adctest = 0x016E

# create access to xylo output
sink   = samna.graph.sink_from(xylo_model.get_source_node())

### 1 Keep both PDM_DATA and PDM_CLK low：(0x2b,1),(0x2d,0),(0x2c,0)


In [5]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,0)
io.write_config(0x2c,0)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)
print(len(events))

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
900
io:	50 uW	AFE core:	13 uW	DFE+SNN core:	493 uW


### 2 Keep PDM_DATA high and PDM_CLK low: (0x2b,1),(0x2d,0),(0x2c,1)


In [6]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,0)
io.write_config(0x2c,1)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
io:	50 uW	AFE core:	13 uW	DFE+SNN core:	493 uW


### 3 Keep PDM_DATA low and PDM_CLK high: (0x2b,1),(0x2d,0),(0x2c,2)


In [7]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,0)
io.write_config(0x2c,2)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
io:	50 uW	AFE core:	13 uW	DFE+SNN core:	492 uW


### 4 Keep PDM_DATA high and PDM_CLK high: (0x2b,1),(0x2d,0),(0x2c,3)


In [8]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,0)
io.write_config(0x2c,3)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
io:	50 uW	AFE core:	13 uW	DFE+SNN core:	492 uW


### 5. Toggle PDM_DATA at some frequency while keeping PDM_CLK low:  (0x2b,1),(0x2d,1),(0x2c,0)


In [9]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,1)
io.write_config(0x2c,0)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
io:	50 uW	AFE core:	13 uW	DFE+SNN core:	492 uW


### 6 Toggle PDM_DATA at some frequency while keeping PDM_CLK high: (0x2b,1),(0x2d,1),(0x2c,1)


In [10]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,1)
io.write_config(0x2c,1)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
io:	51 uW	AFE core:	13 uW	DFE+SNN core:	517 uW


### 7 Keep PDM_DATA low while toggling PDM_CLK at some frequency: (0x2b,1),(0x2d,2),(0x2c,0)

In [11]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,2)
io.write_config(0x2c,0)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
io:	50 uW	AFE core:	13 uW	DFE+SNN core:	517 uW


### 8 Keep PDM_DATA high while toggling PDM_CLK at some frequency: (0x2b,1),(0x2d,2),(0x2c,2)


In [12]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,2)
io.write_config(0x2c,2)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
io:	50 uW	AFE core:	13 uW	DFE+SNN core:	517 uW


### 9 Toggle both PDM_CLK and PDM_DATA at some frequency: (0x2b,1),(0x2d,3)


In [13]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()
xylo_node.get_model().apply_configuration(xylo_config)

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,3)

# read dfe_ctrl
if (enable_main_clock):
    print(hex(read_register(27)))

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
0x100
io:	50 uW	AFE core:	13 uW	DFE+SNN core:	517 uW


In [14]:
print(hex(read_register(pad_ctrl)))


0x222
